In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os

# Agregar la carpeta raíz del proyecto al path
sys.path.append(os.path.dirname(os.getcwd()))

In [2]:
import pandas as pd
print("Cargando datos procesados...")
train_df = pd.read_csv('../temp/train_processed.csv')
test_df = pd.read_csv('../temp/test_processed.csv')

# 2. Separar X e y
X_train = train_df.drop(columns=['popularity'])
y_train = train_df['popularity']

X_test = test_df.drop(columns=['popularity'])
y_test = test_df['popularity']

print(f"X_train: {X_train.shape}")
print(f"X_test: {X_test.shape}")

Cargando datos procesados...
X_train: (333345, 43)
X_test: (83336, 43)


In [3]:
from helpers.trainer import ModelTrainer
trainer = ModelTrainer(
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    y_test=y_test,
    target_col='popularity'
)

✓ ModelTrainer inicializado - Target: popularity
  Train: 333,345 muestras, 43 features
  Test: 83,336 muestras
  Target range: [0.0, 96.0]
  Target mean: 29.3 ± 15.8


In [ ]:
custom_grid = {
    'n_estimators': [200, 300],
    'learning_rate': [0.05, 0.1],
    'max_depth': [5, 7]
} # Pueden configurar esta grid como deseen o quitarla para usar la por defecto

In [4]:
trainer.train_model(
    model_name='xgboost',
    cv=5,
    param_grid=None, # Pueden pasar custom_grid o None para usar la grid por defecto
)


ENTRENANDO: XGBOOST
Grid de búsqueda:
  n_estimators: [100, 200, 300]
  learning_rate: [0.01, 0.05, 0.1]
  max_depth: [3, 5, 7]
  min_child_weight: [1, 3]
  subsample: [0.8, 1.0]
  colsample_bytree: [0.8, 1.0]
  reg_alpha: [0, 0.1]
  reg_lambda: [1, 2]

 Buscando mejores hiperparámetros con 5-Fold CV...
  Total combinaciones: 864

  [1/864] Probando: {'n_estimators': 100, 'learning_rate': 0.01, 'max_depth': 3, 'min_child_weight': 1, 'subsample': 0.8, 'colsample_bytree': 0.8, 'reg_alpha': 0, 'reg_lambda': 1}
    MSE CV: 215.5501 (±1.3891)
    ✓ Nuevo mejor score: 215.5501

  [2/864] Probando: {'n_estimators': 100, 'learning_rate': 0.01, 'max_depth': 3, 'min_child_weight': 1, 'subsample': 0.8, 'colsample_bytree': 0.8, 'reg_alpha': 0, 'reg_lambda': 2}
    MSE CV: 215.5637 (±1.3930)

  [3/864] Probando: {'n_estimators': 100, 'learning_rate': 0.01, 'max_depth': 3, 'min_child_weight': 1, 'subsample': 0.8, 'colsample_bytree': 0.8, 'reg_alpha': 0.1, 'reg_lambda': 1}
    MSE CV: 215.5501 (±1.3

{'model_name': 'xgboost',
 'best_params': {'n_estimators': 300,
  'learning_rate': 0.1,
  'max_depth': 7,
  'min_child_weight': 3,
  'subsample': 0.8,
  'colsample_bytree': 1.0,
  'reg_alpha': 0.1,
  'reg_lambda': 2},
 'cv_score': np.float64(150.51605529785155),
 'train_metrics': {'mse': 134.22781372070312,
  'rmse': np.float64(11.585672778078239),
  'mae': 9.134313583374023,
  'r2': 0.459827721118927},
 'test_metrics': {'mse': 151.5790252685547,
  'rmse': np.float64(12.311743388673868),
  'mae': 9.67824935913086,
  'r2': 0.3921581506729126},
 'training_time': 14202.751884,
 'model': XGBRegressor(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=1.0, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
 

In [5]:
print(trainer.results.keys())  #Debe incluir el modelo entrenado

dict_keys(['xgboost'])


In [7]:
results = trainer.results['xgboost'] #No da error si el modelo fue entrenado correctamente
print(f"XGBoost CV MSE: {results['cv_score']:.4f}")
print(f"XGBoost Test R²: {results['test_metrics']['r2']:.4f}")

XGBoost CV MSE: 150.5161
XGBoost Test R²: 0.3922


In [9]:
trainer.save_model(model_name='xgboost', path='../models/xgboost_model.joblib')

✓ Modelo guardado: ../models/xgboost_model.joblib
✓ Metadata guardada: ../models/xgboost_model_metadata.joblib


In [10]:

from helpers.trainer import load_model
# Para cargar el modelo guardado posteriormente
loaded_model,metadata = load_model(path='../models/xgboost_model.joblib')

loaded_model

✓ Modelo: xgboost
  CV MSE: 150.5161
  Test R²: 0.3922
  Test RMSE: 12.3117


,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,1.0
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None
